In [14]:
using Pkg
Pkg.develop(path="../apuu")

using Revise
using apuu.RobotEnv2D: POMDPscenario, InitParticleBelief, SampleMotionModel, GenerateObservation, ObsLikelihood, scatterParticles!
using apuu.ParticleFilter: ParticleBelief, PropagateParticleBelief, PosteriorParticleBelief, ResampleParticles
using apuu.Paths: HW2_PLOTS

using Revise
using Distributions
using Random
using LinearAlgebra
using Plots
using StatsPlots
using Parameters
using StatsBase


   Resolving package versions...
     Project No packages added to or removed from `~/.julia/environments/v1.12/Project.toml`
    Manifest No packages added to or removed from `~/.julia/environments/v1.12/Manifest.toml`


In [ ]:
# definition of the random number generator with seed
ID = 318803129
rng = MersenneTwister(ID)

# parameters
μ0 = [0.0,0.0]
Σ0 = I(2)
F = I(2)
Σw = 0.1^2 * I(2)
Σv = I(2)
d = 1.0

# 

1.0

In [23]:
# set beacons locations
beacons = Matrix{Float64}(undef, 0, 2)  # 0 rows, 2 columns (empty matrix)

# initialize the scenario
𝒫 = POMDPscenario(F=F, Σw=Σw, Σv=Σv, rng = rng, beacons=beacons, d=d)

# initialize particle belief
n_particles = 10
b0 = InitParticleBelief(𝒫, n_particles, μ0, Σ0)

xgt0 = [-0.5, -0.2] # ground truth initial location
ai = ones(2) # action of the action sequence
T = 10 # time steps

# generate motion trajectory
τ = [xgt0] # trajectory
for i in 0:T-1
    push!(τ, SampleMotionModel(𝒫, ai, τ[end]))
end

@show τ

# generate observation trajectory
τ_obs = []
for i in 1:T
    push!(τ_obs, GenerateObservation(𝒫, τ[i+1]))
end

@show τ_obs

pl_gt = plot([x[1] for x in τ], [x[2] for x in τ], label="gt", markershape=:diamond, title="Trajectory and Observations", color=:black, aspect_ratio=:equal)
scatter!(pl_gt, [z[1] for z in τ_obs], [z[2] for z in τ_obs], label="obs", markershape=:star5, color=:orange)
savefig(pl_gt, joinpath(HW2_PLOTS, "ground_truth_$ID.png"))

# iii. belief propagation (without observation update)
beliefs = Vector{ParticleBelief}(undef, T+1)
beliefs[1] = b0
for t in 1:T
    beliefs[t+1] = PropagateParticleBelief(𝒫, beliefs[t], ai, SampleMotionModel)
end

# plot the actual trajectory and the propagated particle beliefs (with scatterParticles)
pl_beliefs = plot([x[1] for x in τ], [x[2] for x in τ], label="gt", markershape=:diamond, title="Trajectory and Particle Beliefs", color=:black, aspect_ratio=:equal)
for t in 1:T+1
    scatterParticles!(pl_beliefs, beliefs[t], "t=$(t-1)")
end
savefig(pl_beliefs, joinpath(HW2_PLOTS, "particle_beliefs_$ID.png"))

τ = [[-0.5, -0.2], [0.7035225685270402, 0.7380074342548643], [1.9462715523951313, 1.661512207977847], [2.9966353685407974, 2.7682939942114952], [3.833975600086246, 3.9216638523807874], [4.896512644215891, 4.8774184700843035], [5.8836165969880945, 5.841179958087824], [6.764399277693731, 6.831972941536186], [7.867579721130346, 7.751629216669266], [8.845108837208995, 8.560780331848699], [9.836463529572365, 9.538387583642827]]
τ_obs = Any[[1.1459490371428596, 0.16199852317547703], [1.3990824225724285, 2.3848845165543198], [2.1055164200692884, 3.739729458227695], [5.521570113581552, 3.1751331037814214], [6.4895202505385985, 2.880961315676295], [5.138138101783463, 7.002468239063951], [5.317940079793673, 6.355793701389268], [8.227529164546292, 7.771835526310409], [9.64606238719691, 8.904839353835635], [10.925934338026593, 10.81436846058888]]


"/Users/jonathansomer/repos/apuu-hw2/plots/hw2/particle_beliefs_318803129.png"

In [17]:

# iv. belief update with observations
beliefs_upd = Vector{ParticleBelief}(undef, T+1)
beliefs_upd[1] = b0
for t in 1:T
    beliefs_upd[t+1] = PosteriorParticleBelief(𝒫, beliefs_upd[t], ai, τ_obs[t], SampleMotionModel, ObsLikelihood)
end


In [18]:
# plot the actual trajectory and the updated particle beliefs (with scatterParticles)
pl_beliefs_upd = plot([x[1] for x in τ], [x[2] for x in τ], label="gt", markershape=:diamond, title="Trajectory and Updated Particle Beliefs", color=:black, aspect_ratio=:equal)
for t in 1:T+1
    scatterParticles!(pl_beliefs_upd, beliefs_upd[t], "t=$(t-1)")
end
savefig(pl_beliefs_upd, joinpath(HW2_PLOTS, "updated_particle_beliefs_$ID.png"))

"/Users/jonathansomer/repos/apuu-hw2/plots/hw2/updated_particle_beliefs_318803129.png"

In [19]:

# v. belief update with observations and resampling
ess_threshold = n_particles / 2.0  # change to Inf to always resample
beliefs_resamp = Vector{ParticleBelief}(undef, T+1)
beliefs_resamp[1] = b0
for t in 1:T
    b_upd = PosteriorParticleBelief(𝒫, beliefs_resamp[t], ai, τ_obs[t], SampleMotionModel, ObsLikelihood)
    beliefs_resamp[t+1] = ResampleParticles(𝒫, b_upd, ess_threshold)
end


In [20]:
# plot the actual trajectory and the resampled particle beliefs (with scatterParticles)
pl_beliefs_resamp = plot([x[1] for x in τ], [x[2] for x in τ], label="gt", markershape=:diamond, title="Trajectory and Resampled Particle Beliefs", color=:black, aspect_ratio=:equal)
for t in 1:T+1
    scatterParticles!(pl_beliefs_resamp, beliefs_resamp[t], "t=$(t-1)")
end
savefig(pl_beliefs_resamp, joinpath(HW2_PLOTS, "resampled_particle_beliefs_$ID.png"))

"/Users/jonathansomer/repos/apuu-hw2/plots/hw2/resampled_particle_beliefs_318803129.png"